In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/criteo-display-ad-challenge/random_submission.zip


# **NECESSARY IMPORTS**

In [2]:
from __future__ import absolute_import, division, print_function, unicode_literals

# miscellaneous
import builtins
import functools
# import bisect
# import shutil
import time
import json

# numpy
import numpy as np

# onnx
# The onnx import causes deprecation warnings every time workers
# are spawned during testing. So, we filter out those warnings.
import warnings
with warnings.catch_warnings():
    warnings.filterwarnings("ignore", category=DeprecationWarning)
import onnx

# pytorch
import torch
import torch.nn as nn
from torch.nn.parallel.parallel_apply import parallel_apply
from torch.nn.parallel.replicate import replicate
from torch.nn.parallel.scatter_gather import gather, scatter
import sklearn.metrics

# from torchviz import make_dot
# import torch.nn.functional as Functional
# from torch.nn.parameter import Parameter

from torch.optim.lr_scheduler import _LRScheduler

##**Data Generation** : 
import dlrm_data_pytorch as dp

Trying to declare it here

#**Declaring Embedding Functions**:


**quotient-remainder trick**
from tricks.qr_embedding_bag import QREmbeddingBag
**mixed-dimension trick**
from tricks.md_embedding_bag import PrEmbeddingBag, md_solver


**PrEmbeddingBag** : Mixed Embedding Generation

In [3]:
from __future__ import absolute_import, division, print_function, unicode_literals
import torch
import torch.nn as nn


def md_solver(n, alpha, d0 = None, B = None, round_dim = True, k = None):

  """
  An external facing function call for mixed-dimension assignment with the 
  alpha power temperature heuristic

  Inputs : 
  n           (torch.LongTensor): Vector of num of rows for each embedding matrix
  alpha       (torch.FloatTensor) : Scalar, non-negative, controls dim, skew
  d0          (torch.FloatTensor) : Scalar, baseline embedding dimension
  B           (torch.FloatTensor) : Scalar, parameter budget for embedding layer
  round_dim   (bool) : Flag for rounding dims to nearest pow of 2
  k           (torch.LongTensor): Vector of average number of queries per inference
  
  """

  n, indices = torch.sort(n)
  k = k[indices] if k is not None else torch.ones(len(n))
  d = alpha_power_rule(n.type(torch.float) / k, alpha, d0=d0, B = B)

  if round_dim:
    d = pow_2_round(d)

  undo_sort = [0] * len(indices)
  
  for i, v in enumerate(indices):
    undo_sort[v] = i

  return d[undo_sort]

def alpha_power_rule(n, alpha, d0 = None, B=None):

  if d0 is not None: 
    lamb = d0 * ( n[0].type(torch.float) ** alpha)

  elif B is not None:
    lamb = B / torch.sum(n.type(torch.float) ** (1 - alpha))

  else : 
    raise ValueError("Must specify either d0 or B")

  d = torch.ones(len(n)) * lamb * (n.type(torch.float) ** (-alpha))

  for i in range(len(d)):

    if i == 0 and d0 is not None:
      d[i] = d0

    else : 
      d[i] = 1 if d[i] <1 else d[i]

  return (torch.round(d).type(torch.long))

def pow_2_round(dims):

  return 2 ** torch.round(torch.log2(dims.type(torch.float)))


class PrEmbeddingBag(nn.Module):

  # torch.nn.Module : Base class for all neural network modules
  # Our model should  subclass this class. 
  # Can contain other Modules, allowing to nest them in a tree structure. 
  # We can assign submodules as regular attributes
  # Submodules assigned this way will be registered , and will have their parameters converted too 
  # when you call 'to()'

  def __init__(self, num_embeddings, embedding_dim, base_dim):

    super(PrEmbeddingBag, self).__init__()
    self.embs = nn.EmbeddingBag(
        num_embeddings, embedding_dim, mode = "sum", sparse = True
    )

    # super() function is used to give access to methods and properties of a parent or 
    # sibling class. 
    # super() function returns an object that represents the parent class.

    torch.nn.init.xavier.uniform(self.embs.weight)

    if embedding_dim < base_dim:
      self.proj = nn.Linear(embedding_dim, base_dim, bias = False)
      torch.nn.init.xavier_uniform_(self.proj.weight)
    
    elif embedding_dim = base_dim:
      self.proj = nn.Identity() # Placeholder operator that is Argument-Insensitive

    else : 
      raise ValueError(
          "Embedding dim" + str(embedding_dim) + "> base dim" + str(base_dim)
      )

  def forward(self, input, offsets=None, per_sample_weights = None):
    return self.proj(self.embs(
        input, 
        offsets = offsets,
        per_sample_weights = per_sample_weights
    ))

          

SyntaxError: invalid syntax (<ipython-input-3-95d81fad0afe>, line 90)

**Quotient Remainder Embedding Trick**

In [4]:
from __future__ import absolute_import, division, print_function, unicode_literals
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.nn.parameter import Parameter
import numpy as np

class QREmbeddingBag(nn.Module):

  r"""Computes sum or mean over 2 'bags' of embeddings, one using the quotient of the indices and the
      other using the remainder of the indices, without instantiating the intermediate embeddings, then 
      performs an operation to combine these.

      For bags of constant length and no :attr: 'per_sample_weights', this class

      * with ''mode="sum"'' is equivalent to :class:'-torch.nn.Embedding' followed by ''torch.sum(dim=0)'',
      * with ''mode="mean"'' is equivalent to :class:'-torch.nn.Embedding' followed by ''torch.sum(dim=0)'',
      * with ''mode="max"'' is equivalent to :class:'-torch.nn.Embedding' followed by ''torch.sum(dim=0)''

      However, :class:'-torch.nn.EmbeddingBag is much more time and memory efficient than using a chain of 
      these operations. 

      QREmbeddingBag also supports per-sample weights as an argument to the forward pass. 
      This scales the output of the Embedding before performing a weighted reduction as specified 
      by ''mode''. If :attr: 'per_sample_weights'' is passed, the only supported ''mode'' is 
      ''sum'', which computes a weighted sum according to :attr:'per_sample_weights.

      KNOWN ISSUES: 
      Autograd breaks with multiple GPUs. It breaks only with multiple embeddings. 

      Args :

        num_categories (int) : total number of unique categories. The input indices must be in 0, 1,...
                               ,num_categories - 1.
        embedding_dim (list) : list of sizes for each embedding vector in each table. If ''add''
                               or ''mult'' operation are used, these embedding dimensions must be the same. 
                               If a single embedding_dim is used, then it will use this embedding_dim 
                               for both embedding tables. 

        num_collisions (int) : number of collisions to enforce.

        operation (string, optional): "concat", "add", or "mult" . Specifies the operation to compose embeddings
                                      , "concat" concatenates embeddings, "add" sums the embeddings, and
                                      "mult" multiplies (component-wise) the embeddings.
                                      Default : 'mult'

       max_norm (float, optional) : If given, each embedding vector with norm larger than :attr: 
                                    max_norm is renormalized to have norm :attr: 'max_norm'
        norm_type (float, optional): The p of the p-norm to compute for the :attr:`max_norm` option. Default ``2``.
        scale_grad_by_freq (boolean, optional): if given, this will scale gradients by the inverse of frequency of
                                                the words in the mini-batch. Default ``False``.
                                                Note: this option is not supported when ``mode="max"``.
        mode (string, optional): ``"sum"``, ``"mean"`` or ``"max"``. Specifies the way to reduce the bag.
                                 ``"sum"`` computes the weighted sum, taking :attr:`per_sample_weights`
                                 into consideration. ``"mean"`` computes the average of the values
                                 in the bag, ``"max"`` computes the max value over each bag.
                                 Default: ``"mean"``
        sparse (bool, optional): if ``True``, gradient w.r.t. :attr:`weight` matrix will be a sparse tensor. See
                                 Notes for more details regarding sparse gradients. Note: this option is not
                                 supported when ``mode="max"``.
    Attributes:
        weight (Tensor): the learnable weights of each embedding table is the module of shape
                         `(num_embeddings, embedding_dim)` initialized using a uniform distribution
                         with sqrt(1 / num_categories).
    Inputs: :attr:`input` (LongTensor), :attr:`offsets` (LongTensor, optional), and
        :attr:`per_index_weights` (Tensor, optional)
        - If :attr:`input` is 2D of shape `(B, N)`,
          it will be treated as ``B`` bags (sequences) each of fixed length ``N``, and
          this will return ``B`` values aggregated in a way depending on the :attr:`mode`.
          :attr:`offsets` is ignored and required to be ``None`` in this case.
        - If :attr:`input` is 1D of shape `(N)`,
          it will be treated as a concatenation of multiple bags (sequences).
          :attr:`offsets` is required to be a 1D tensor containing the
          starting index positions of each bag in :attr:`input`. Therefore,
          for :attr:`offsets` of shape `(B)`, :attr:`input` will be viewed as
          having ``B`` bags. Empty bags (i.e., having 0-length) will have
          returned vectors filled by zeros.
        per_sample_weights (Tensor, optional): a tensor of float / double weights, or None
            to indicate all weights should be taken to be ``1``. If specified, :attr:`per_sample_weights`
            must have exactly the same shape as input and is treated as having the same
            :attr:`offsets`, if those are not ``None``. Only supported for ``mode='sum'``.
    Output shape: `(B, embedding_dim)`
    """

    __constants__ = ['num_categories', 'embedding_dim', 'num_collisions',
                     'operation', 'max_norm', 'norm_type','scale_grad_by_freq',
                     'mode','sparse']

    
    def __init__(self, num_categories, embedding_dim, num_collisions,
                 operation='mult', max_norm=None, norm_type=2.,
                 scale_grad_by_freq=False, mode='mean', sparse=False,
                 _weight=None):
      super(QREmbeddingBag, self).__init__()

      assert operation in ['concat', 'mult', 'add'], 'Not valid operation!'

      self.num_categories = num_categories
      
      if isinstance(embedding_dim, int) or len(embedding_dim) == 1:
        self.embedding_dim = [embedding_dim, embedding_dim]

      else:
        self.embedding_dim = embedding_dim

      self.num_collisions = num_collisions
      self.operation = operation
      self.max_norm = max_norm
      self.norm_type = norm_type
      self.scale_grad_by_freq = scale_grad_by_freq

      if self.operation == 'add' or self.operation == 'mult':

        assert self.embedding_dim[0] == self.embedding_dim[1], \
        'Embedding Dimensions do not match'

      self.num_embeddings = [int(np.cell(num_categories / num_collisions)),
                             num_collisions]

      if _weight is None:

        self.weight_q = Parameter(torch.Tensor(self.num_embeddings[0], self.embedding_dim[0]))
        self.weight_r = Parameter(torch.Tensor(self.num_embeddings[1], self.embedding_dim[1]))
        self.reset_parameters()

      else : 
        assert list(_weight[0].shape) == [self.num_embeddings[0], self.embedding_dim[0]], \
        'Shape of weight for quotient table does not match num_embeddings and embedding_dim'

        assert list(_weight[1].shape) == (self.num_embeddings[1], self.embedding_dim[1]), \
        'Shape of weight for remainder table does not match num_embeddings and embedding_dim'

        self.weight_q = Parameter(_weight[0])
        self.weight_r = Parameter(_weight[1])

      self.mode = mode
      self.sparse = sparse


    def reset_parameters(self):

      nn.init.uniform_(self.weight_q_, np.sqrt(1 / self.num_categories))
      nn.init.uniform (self.weight_r, np.sqrt(1 / self.num_categories))


    def forward(self. input, offsets = None, per_sample_weights = None):

      input_q = (input / self,num_collisions).long()
      input_r = torch.remainder(input, self.num_collisions).long()

      embed_q = F.embedding_bag(input_q, self.weight_q, offsets, self.max_norm,
                                self.norm_type, self.scale_grad_by_freq, self.mode,
                                self.sparse, per_sample_weights)
      embed_r = F.embedding_bag(input_r, self.weight_r, offsets, sekf.max_norm,
                                self.norm_type, self.scale_grad_by_freq, self.model,
                                self.sparse, per_sample_weights)

      if self.operation == 'concat':
        embed = torch.cat((embed_q, embed_r), dim=1)

      elif self.operation == 'add':
        embed = embed_q + embed_r

      elif self.operation == 'mult':
        embed = embed_q * embed_r

      return embed


    def extra_repr(self):

      s = '{num_embeddings}, {embedding_dim}'

      if self.max_norm is not None:
        s += ',max_norm = {max_norm}'

      if self.norm_type != 2:
        s += ', norm_type = {norm_type}'

      if self.scale_grad_by_freq is not False:
        s += ',scale_grad_by_freq = {scale_grade_by_freq}'

      s += ', mode={mode}'
      return s.format(**self.__dict__)    

IndentationError: unexpected indent (<ipython-input-4-a6b2e39f286e>, line 85)

# # # **CREATING EMBEDDING FUNCTION**

In [5]:
def create_emb (self, m, ln):
    
    emb_l = nn.ModuleList()
    for i in range(0, ln.size):
        n = ln[i]
        
        # Construct Embedding Operator
        
        if self.qr_flag and n > self.qr_threshold:
            # Using Quotient-Remainder Embedding
            EE = QREmbeddingBag(n, m, self.qr_collisions,
                               operation = self.qr_operation, mode = "sum", sparse = True)
            
            
        elif self.md_flag:
            # Using Mixed Dimension Embedding
            base = max(m)
            _m = m[i] if n > self.md_threshold else base
            EE = PrEmbeddingBag(n, _m, base)
            # use np initialization as below for consistency...
            W = np.random.uniform(
                            low = -np.sqrt(1/n), high = np.sqrt(1/n), size = (n, _m)
            ).astype(np.float32)
            
            EE.embs.weight.data = torch.tensor(W, requires_grad = True)
            
        else:
            # Using Normal Embedding
            '''
            Computes sum or mean of bag of embeddings, without instantiating the intermediate embeddings
            EmbeddingBag is much more time and memory efficient that using a chain of these operations
            '''
            EE = nn.EmbeddingBag(n, m, mode = "sum", sparse = True)
            
            # Initialize Embeddings via XAVIER INITIALIZATION
            # nn.init.uniform_(EE.weight, a = -np.sqrt(1/n), b = np.sqrt(1/n))
            W = np.random.uniform(
                        low=-np.sqrt(1 / n), high=np.sqrt(1 / n), size=(n, m)
                                ).astype(np.float32)
            # approach 1
            EE.weight.data = torch.tensor(W, requires_grad=True)
            # approach 2
            # EE.weight.data.copy_(torch.tensor(W))
            # approach 3
            # EE.weight = Parameter(torch.tensor(W),requires_grad=True)

        emb_l.append(EE)

      return emb_l


def apply_emb(self, lS_o, lS_i, emb_l):
    
    
    # emb_l : Returned by create_emb
    
    # WARNING : Notice that we are processing batch at once
    # We implicitly assume that data is laid out such that : 
    # 1. Each embedding is indexed with a group of sparse indices,
    #    corresponding to a single lookup
    # 2. For each Embedding the lookups are further organized into a batch
    # 3. For a list of Embedding tables there is a list of batched lookups
    
    ly = []
    # for k, sparse_index_group_batch in enumerate(lS_i):
    for k in range(len(lS_i)):
        
        sparse_index_group_batch = lS_i[k]
        sparse_offset_group_batch = lS_o[k]
        
        # Offset is simply character location within that file , usually starting with 0:
        # thus offset 240 is actually 241st byte in the file
        
        # Embedding Lookup
        # We are using Embedding Bag, which implicitly uses sum operator
        # The embeddings are represented as tall matrices, with sum
        # happening vertically across 0 axis, resulting in a row vector
        E = emb_l[k]
        V = E(sparse_index_group_batch, sparse_offset_group_batch)
        
        ly.append(V)
        
    # print(ly)
    return ly

IndentationError: unindent does not match any outer indentation level (<tokenize>, line 49)

# **Multi-Layer Perceptron**

In [7]:
class DLRM_Net(nn.Module):
    
    def create_mlp(self, ln, sigmoid_layer):
        
        # BUILD MLP layer by layer
        layers = nn.ModuleList() # Holding layers as submodules in a list format
        
        for i in range(0, ln.size - 1):
            
            n = ln[i]
            m = ln[i+1]
            
            # Construct fully connected operator
            LL = nn.Linear(int(n), int(m), bias = True)
            
            mean = 0.0
            std_dev = np.sqrt(2 / (m+n))
            W = np.random.normal(mean, std_dev, size = (m, n)).astype(np.float32)
            
            std_dev = np.sqrt(1/ m)
            bt = np.random.normal(mean, std_dev, size = m).astype(np.float32)
            
            LL.weight.data = torch.tensor(W, requires_grad = True)
            LL.bias.data = torch.tensor(bt, requires_grad = True)
            
            layers.append(LL)
            
            if i == sigmoid_layer:
                
                layers.append(LL)
                
            else:
                layers.append(nn.ReLU())
                
            return torch.nn.Sequential(*layers)
        
        
    def apply_mlp(self, x, layers):
        
        # Approach 1 : use ModuleList
        # for layer in layers:
        #        x = layer(x)
        # return x
        
        # Approach 2 : Use Sequential container to wrap all layers
        return layers(x)

# **FACTORIZATION MACHINES**

Checking interaction of different features

In [8]:
def interact_features(self, x, ly):
    
    if self.arch_interaction_op == "dot":
        
        # Concatenate dense and sparse features
        (batch_size, d) = x.shape
        T = torch.cat([x] + ly, dim = 1).view(batch_size, -1, d)
        
        ### Perform a dot product
        Z = torch.bmm(T, torch.transpose(T, 1, 2))
        
        # torch.bmm : Performs a batch matrix-matrix product of matrices
        
        # Append dense feature with the interaction sinto a row vector
        
        # Approach 1 : all
        # Zflat = Z.view(batch_size, -1)
        
        # Appraoch 2 : unique
        _, ni, nj = Z.shape
        
         # Approach 1 : tril_indices 
    # offset = 0 if self.arch_interaction_itself else -1
    # li, lj - torch.tril_indices (ni, nj, offset = offset)
    
        # Approach 2 : Custom 
        offset = 1 if self.arch_interaction_itself else 0
        
        li = torch.tensor([i for i in range(ni) for j in range(i+offset)])
        lj = torch.tensor([ j for i in range(nj) for j in range(i + offset)])
        Zflat = Z[:, li, lj]
        
        # Concatenate dense features and interactions 
        R = torch.cat([x] + [ZFlat], dim = 1)
        
    elif self.arch_interaction_op == "cat":
        # Concatenate features into a row vector
        R = torch.cat([x] + ly, dim = 1)
        
    else : 
        sys.exit(
        "ERROR: --arch-interactopn-op="
        + self.arch_interaction_op
        + " is not supported"
        )
        
    return R



# **SEQUENTIAL AND PARALLEL EXECUTION**

Depending on number of devices

# # For simplified workflow of events follow sequential execution

In [9]:
def forward(self, dense_x, lS_o, lS_i):
    
    if self.ndevices <= 1:
        return self.sequential_forward(dense_x, lS_o, lS_i)
    
    else :
        return self.parallel_forward(dense_x, lS_o, lS_i)
    
def sequential_forward (self, dense_x, lS_o, lS_i):
    
    # process dense features ( using bottom mlp), resulting in a row vector
    x = self.apply_mlp(dense_x, self.bot_l)
    
    # process sparse features (using embeddings), resulting in a list of row vectors
    ly = self.apply_emb(lS_o, lS_i, self.emb_l)
    
    z = self.interact_features(x, ly)
    
    # obtain probability of a click using top mlp
    p = self.apply_mlp(z, self.top_l)
    
    # Clamp output if needed
    if 0.0 < self.loss_threshold and self.loss_threshold < 1.0 : 
        z = torch.clamp(p, min = self.loss_threshold, max = (1.0 - self.loss_threshold))
        
    else : 
        z= p
        
    return z


def parallel_forward(self, dense_x, lS_o, lS_i):
    ### prepare model (overwrite) ###
    # WARNING: # of devices must be >= batch size in parallel_forward call
    batch_size = dense_x.size()[0]
    ndevices = min(self.ndevices, batch_size, len(self.emb_l))
    device_ids = range(ndevices)
    # WARNING: must redistribute the model if mini-batch size changes(this is common
    # for last mini-batch, when # of elements in the dataset/batch size is not even
    if self.parallel_model_batch_size != batch_size:
        self.parallel_model_is_not_prepared = True

    if self.parallel_model_is_not_prepared or self.sync_dense_params:
        # replicate mlp (data parallelism)
        self.bot_l_replicas = replicate(self.bot_l, device_ids)
        self.top_l_replicas = replicate(self.top_l, device_ids)
        self.parallel_model_batch_size = batch_size

    if self.parallel_model_is_not_prepared:
        # distribute embeddings (model parallelism)
        t_list = []
        for k, emb in enumerate(self.emb_l):
            d = torch.device("cuda:" + str(k % ndevices))
            emb.to(d)
            t_list.append(emb.to(d))
        self.emb_l = nn.ModuleList(t_list)
        self.parallel_model_is_not_prepared = False

    ### prepare input (overwrite) ###
    # scatter dense features (data parallelism)
    # print(dense_x.device)
    dense_x = scatter(dense_x, device_ids, dim=0)
    # distribute sparse features (model parallelism)
    if (len(self.emb_l) != len(lS_o)) or (len(self.emb_l) != len(lS_i)):
        sys.exit("ERROR: corrupted model input detected in parallel_forward call")

    t_list = []
    i_list = []
    for k, _ in enumerate(self.emb_l):
        d = torch.device("cuda:" + str(k % ndevices))
        t_list.append(lS_o[k].to(d))
        i_list.append(lS_i[k].to(d))
    lS_o = t_list
    lS_i = i_list

    ### compute results in parallel ###
    # bottom mlp
    # WARNING: Note that the self.bot_l is a list of bottom mlp modules
    # that have been replicated across devices, while dense_x is a tuple of dense
    # inputs that has been scattered across devices on the first (batch) dimension.
    # The output is a list of tensors scattered across devices according to the
    # distribution of dense_x.
    x = parallel_apply(self.bot_l_replicas, dense_x, None, device_ids)
    # debug prints
    # print(x)

    # embeddings
    ly = self.apply_emb(lS_o, lS_i, self.emb_l)
    # debug prints
    # print(ly)

    # butterfly shuffle (implemented inefficiently for now)
    # WARNING: Note that at this point we have the result of the embedding lookup
    # for the entire batch on each device. We would like to obtain partial results
    # corresponding to all embedding lookups, but part of the batch on each device.
    # Therefore, matching the distribution of output of bottom mlp, so that both
    # could be used for subsequent interactions on each device.
    if len(self.emb_l) != len(ly):
        sys.exit("ERROR: corrupted intermediate result in parallel_forward call")

    t_list = []
    for k, _ in enumerate(self.emb_l):
        d = torch.device("cuda:" + str(k % ndevices))
        y = scatter(ly[k], device_ids, dim=0)
        t_list.append(y)
    # adjust the list to be ordered per device
    ly = list(map(lambda y: list(y), zip(*t_list)))
    # debug prints
    # print(ly)

    # interactions
    z = []
    for k in range(ndevices):
        zk = self.interact_features(x[k], ly[k])
        z.append(zk)
    # debug prints
    # print(z)

    # top mlp
    # WARNING: Note that the self.top_l is a list of top mlp modules that
    # have been replicated across devices, while z is a list of interaction results
    # that by construction are scattered across devices on the first (batch) dim.
    # The output is a list of tensors scattered across devices according to the
    # distribution of z.
    p = parallel_apply(self.top_l_replicas, z, None, device_ids)

    ### gather the distributed results ###
    p0 = gather(p, self.output_d, dim=0)

    # clamp output if needed
    if 0.0 < self.loss_threshold and self.loss_threshold < 1.0:
        z0 = torch.clamp(
            p0, min=self.loss_threshold, max=(1.0 - self.loss_threshold)
        )
    else:
        z0 = p0

    return z0


# # DATA PREPROCESSING : CRITEO DATASET

In [10]:
from __future__ import absolute_import, division. print_function, unicode_literals

# others

from os import path
import bisect
import collections

import data_utils

# numpy 
import numpy as np
from numpy import random as ra

# PyTorch
import torch
from torch.utils.data import Dataset, RandomSampler

import data_loader_terabyte

SyntaxError: invalid syntax (<ipython-input-10-551895b8c07f>, line 1)

In [11]:
def make_criteo_data_and_loaders(args):


  # called from DATA_S_PYTORCH

    if args.mlperf_logging and args.memory_map and args.data_set == "terabyte":
        # more efficient for larger batches
        data_directory = path.dirname(args.raw_data_file)

        if args.mlperf_bin_loader:
            lstr = args.processed_data_file.split("/")
            d_path = "/".join(lstr[0:-1]) + "/" + lstr[-1].split(".")[0]
            train_file = d_path + "_train.bin"
            test_file = d_path + "_test.bin"
            # val_file = d_path + "_val.bin"
            counts_file = args.raw_data_file + '_fea_count.npz'

            if any(not path.exists(p) for p in [train_file,
                                                test_file,
                                                counts_file]):
                ensure_dataset_preprocessed(args, d_path)

            train_data = data_loader_terabyte.CriteoBinDataset(
                data_file=train_file,
                counts_file=counts_file,
                batch_size=args.mini_batch_size,
                max_ind_range=args.max_ind_range
            )

            train_loader = torch.utils.data.DataLoader(
                train_data,
                batch_size=None,
                batch_sampler=None,
                shuffle=False,
                num_workers=0,
                collate_fn=None,
                pin_memory=False,
                drop_last=False,
                sampler=RandomSampler(train_data) if args.mlperf_bin_shuffle else None
            )

            test_data = data_loader_terabyte.CriteoBinDataset(
                data_file=test_file,
                counts_file=counts_file,
                batch_size=args.test_mini_batch_size,
                max_ind_range=args.max_ind_range
            )

            test_loader = torch.utils.data.DataLoader(
                test_data,
                batch_size=None,
                batch_sampler=None,
                shuffle=False,
                num_workers=0,
                collate_fn=None,
                pin_memory=False,
                drop_last=False,
            )
        else:
            data_filename = args.raw_data_file.split("/")[-1]

            train_data = CriteoDataset(
                args.data_set,
                args.max_ind_range,
                args.data_sub_sample_rate,
                args.data_randomize,
                "train",
                args.raw_data_file,
                args.processed_data_file,
                args.memory_map,
                args.dataset_multiprocessing
            )

            test_data = CriteoDataset(
                args.data_set,
                args.max_ind_range,
                args.data_sub_sample_rate,
                args.data_randomize,
                "test",
                args.raw_data_file,
                args.processed_data_file,
                args.memory_map,
                args.dataset_multiprocessing
            )

            train_loader = data_loader_terabyte.DataLoader(
                data_directory=data_directory,
                data_filename=data_filename,
                days=list(range(23)),
                batch_size=args.mini_batch_size,
                max_ind_range=args.max_ind_range,
                split="train"
            )

            test_loader = data_loader_terabyte.DataLoader(
                data_directory=data_directory,
                data_filename=data_filename,
                days=[23],
                batch_size=args.test_mini_batch_size,
                max_ind_range=args.max_ind_range,
                split="test"
            )
    else:
        train_data = CriteoDataset(
            args.data_set,
            args.max_ind_range,
            args.data_sub_sample_rate,
            args.data_randomize,
            "train",
            args.raw_data_file,
            args.processed_data_file,
            args.memory_map,
            args.dataset_multiprocessing
        )

        test_data = CriteoDataset(
            args.data_set,
            args.max_ind_range,
            args.data_sub_sample_rate,
            args.data_randomize,
            "test",
            args.raw_data_file,
            args.processed_data_file,
            args.memory_map,
            args.dataset_multiprocessing
        )

        train_loader = torch.utils.data.DataLoader(
            train_data,
            batch_size=args.mini_batch_size,
            shuffle=False,
            num_workers=args.num_workers,
            collate_fn=collate_wrapper_criteo,
            pin_memory=False,
            drop_last=False,  # True
        )

        test_loader = torch.utils.data.DataLoader(
            test_data,
            batch_size=args.test_mini_batch_size,
            shuffle=False,
            num_workers=args.test_num_workers,
            collate_fn=collate_wrapper_criteo,
            pin_memory=False,
            drop_last=False,  # True
        )

    return train_data, train_loader, test_data, test_loader


# uniform ditribution (input data)
class RandomDataset(Dataset):

    def __init__(
            self,
            m_den,
            ln_emb,
            data_size,
            num_batches,
            mini_batch_size,
            num_indices_per_lookup,
            num_indices_per_lookup_fixed,
            num_targets=1,
            round_targets=False,
            data_generation="random",
            trace_file="",
            enable_padding=False,
            reset_seed_on_access=False,
            rand_seed=0
    ):
        # compute batch size
        nbatches = int(np.ceil((data_size * 1.0) / mini_batch_size))
        if num_batches != 0:
            nbatches = num_batches
            data_size = nbatches * mini_batch_size
            # print("Total number of batches %d" % nbatches)

        # save args (recompute data_size if needed)
        self.m_den = m_den
        self.ln_emb = ln_emb
        self.data_size = data_size
        self.num_batches = nbatches
        self.mini_batch_size = mini_batch_size
        self.num_indices_per_lookup = num_indices_per_lookup
        self.num_indices_per_lookup_fixed = num_indices_per_lookup_fixed
        self.num_targets = num_targets
        self.round_targets = round_targets
        self.data_generation = data_generation
        self.trace_file = trace_file
        self.enable_padding = enable_padding
        self.reset_seed_on_access = reset_seed_on_access
        self.rand_seed = rand_seed


NameError: name 'Dataset' is not defined

In [12]:
Kaggle Display Advertising Challenge Dataset
# dataset (str): name of dataset (Kaggle or Terabyte)
# randomize (str): determines randomization scheme
#            "none": no randomization
#            "day": randomizes each day"s data (only works if split = True)
#            "total": randomizes total dataset
# split (bool) : to split into train, test, validation data-sets
class CriteoDataset(Dataset):

    def __init__(
            self,
            dataset,
            max_ind_range,
            sub_sample_rate,
            randomize,
            split="train",
            raw_path="",
            pro_data="",
            memory_map=False,
            dataset_multiprocessing=False
    ):
        # dataset
        # tar_fea = 1   # single target
        den_fea = 13  # 13 dense  features
        # spa_fea = 26  # 26 sparse features
        # tad_fea = tar_fea + den_fea
        # tot_fea = tad_fea + spa_fea
        if dataset == "kaggle":
            days = 7
            out_file = "kaggleAdDisplayChallenge_processed"
        elif dataset == "terabyte":
            days = 24
            out_file = "terabyte_processed"
        else:
            raise(ValueError("Data set option is not supported"))
        self.max_ind_range = max_ind_range
        self.memory_map = memory_map

        # split the datafile into path and filename
        lstr = raw_path.split("/")            # Splitting dataset into file
        self.d_path = "/".join(lstr[0:-1]) + "/"    
        self.d_file = lstr[-1].split(".")[0] if dataset == "kaggle" else lstr[-1]
        self.npzfile = self.d_path + (
            (self.d_file + "_day") if dataset == "kaggle" else self.d_file
        )  # Creating npz file 
        self.trafile = self.d_path + (
            (self.d_file + "_fea") if dataset == "kaggle" else "fea"
        )

        # check if pre-processed data is available
        data_ready = True
        if memory_map:

          # memory_map : Takes input from user while executing data_s_pytorch
            for i in range(days):
                reo_data = self.npzfile + "_{0}_reordered.npz".format(i)
                
                if not path.exists(str(reo_data)):
                    data_ready = False
        else:
            if not path.exists(str(pro_data)):
                data_ready = False

        # pre-process data if needed
        # WARNNING: when memory mapping is used we get a collection of files

        if data_ready:
            print("Reading pre-processed data=%s" % (str(pro_data)))
            file = str(pro_data)

        else:
            print("Reading raw data=%s" % (str(raw_path)))
            file = data_utils.getCriteoAdData(
                raw_path,
                out_file,
                max_ind_range,
                sub_sample_rate,
                days,
                split,
                randomize,
                dataset == "kaggle",
                memory_map,
                dataset_multiprocessing
            )

        # get a number of samples per day
        total_file = self.d_path + self.d_file + "_day_count.npz"
        with np.load(total_file) as data:
            total_per_file = data["total_per_file"]


        # Taken from Google :::::
        # An offset into a file is simply the character location within that file, 
        # usually starting with 0; thus "offset 240" is actually the 241st byte in the file.
        # compute offsets per file

        self.offset_per_file = np.array([0] + [x for x in total_per_file])

        for i in range(days):
            self.offset_per_file[i + 1] += self.offset_per_file[i]

        # print(self.offset_per_file)

        # setup data
        if memory_map:
            # setup the training/testing split
            self.split = split
            if split == 'none' or split == 'train':
                self.day = 0
                self.max_day_range = days if split == 'none' else days - 1
            elif split == 'test' or split == 'val':
                self.day = days - 1
                num_samples = self.offset_per_file[days] - \
                              self.offset_per_file[days - 1]
                self.test_size = int(np.ceil(num_samples / 2.))
                self.val_size = num_samples - self.test_size
            else:
                sys.exit("ERROR: dataset split is neither none, nor train or test.")
                
                 # dense feature
                lX.append(Xt)
                # sparse feature (sparse indices)
                lS_offsets.append(lS_emb_offsets)
                lS_indices.append(lS_emb_indices)

                # generate a batch of target (probability of a click)
                P = generate_random_output_batch(n, num_targets, round_targets)
                lT.append(P)

            return (nbatches, lX, lS_offsets, lS_indices, lT)


SyntaxError: invalid syntax (<ipython-input-12-17784370b964>, line 1)